# Chromatin Differentiation in Dysfunctional T cells

In [ ]:
# imports
from datascience import Table
%matplotlib inline
import numpy as np

All data in this lab comes from https://www.nature.com/articles/nature22367. This paper, titled 'Chromatin states define tumour-specific T cell dysfunction and reprogramming', collects ATAC-seq data from normal and dysfunctional T cells that were immunized with Listeria monoctgogenes strain. Here, we compare the accessibility between these two groups. Today, we will look at this datasets to draw similar conclusions regarding the change in epigenetic state of T cell dysfunction.

Data can be found at https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE89309.

First, load in the data.

In [ ]:
table = Table.read_table('https://raw.githubusercontent.com/data-8/mcb-88-connector/gh-pages/data/lab4/dysfunctional_ATACseq_timeseries.csv')
table

In this dataset, each peak has a location (chr, start and end) as well as an associated gene symbol. The peak annotation tells us where the peak occurs.

The rest of the dataset has labels N1, E5, E7, ... L60_3. These abbreviations stand for the following:

<h3>Normal T Cells</h3> 

- N1: Naive T cell
- E5: Effector cell after incubated for 5 days
- E7: Effector cell after incubated for 7 days
- M1: Memory cell

<h3>Dysfunctional T Cells</h3> 

- L5: Dysfunctional T cells after 5 days
- L7: Dysfunctional T cells after 7 days
- L14: Dysfunctional T cells after 14 days
- L21: Dysfunctional T cells after 21 days
- L28: Dysfunctional T cells after 28 days
- L35: Dysfunctional T cells after 35 days
- L60: Dysfunctional T cells after 60 days

You may notice there are multiple samples for each category (i.e. E5_1, E5_2, E5_3). These represent biological replicates.


<h2 style="color:red">** Question 1**</h2> 

How many ATAC-seq peaks does this dataset have?

In [ ]:
# Answer here


<h2 style="color:red">** Question 2**</h2> 

Is there more or less ATAC-seq peaks in this dataset than you would expect in the same dataset for RNA-seq? Explain.

**Answer here:**


## Part 1: Exploring differential accessibility in normal T Cells


For part 1 of the lab, we will compare differentially accessible regions between different T cell activation states in normal T cells. To do this, we need a method that can calculate whether the value of a gene in two cell states is statistically significant. To determine whether sites are significantly different, we need a **peak calling** algorithm. These algorithms are able to determine which genes are significantly differentially accessible between two states (i.e. naive and effector). We will use our own peak calling algorithm to calculate differential peaks.

First, let us define a peak calling function called **callPeaks()**:

In [ ]:
def callPeaks(table, CAT1_NAMES, CAT2_NAMES):
    '''
    Calls peaks on two categories of peak calling data. Can be used for ATAC-seq or RNA-seq.
    Borrowed from http://dept.stat.lsa.umich.edu/~kshedden/Python-Workshop/gene_expression_comparison.html

    param1: Table data: takes for from 
    https://raw.githubusercontent.com/data-8/mcb-88-connector/gh-pages/data/lab4/dysfunctional_ATACseq_timeseries.csv
    param1: column names of first category
    param2: column names of second category
    return: Table of differentially called regions
        Positive values mean the peak was siginificantly higher in the first category
        Negative values mean the peak was significantly higher in the second category
    '''

    # gene list
    GID = list(table.column('symbol'))

    # matrix of values
    dropped = table.drop(['chr', 'start', 'end', 'symbol','refseqID', 'peak_annotation'])
    X = np.array(dropped.to_df().as_matrix())
    # list of categories
    SID = list(table.column_labels)[6:]
    
    # get indices of category names
    CAT1 = [list(dropped.column_labels).index(i) for i in CAT1_NAMES]
    CAT2 = [list(dropped.column_labels).index(i) for i in CAT2_NAMES]
    

    ## Convert X to log scale
    XL = np.log(X) / np.log(2)

    M1 = XL[:,CAT1].mean(1) ## Mean of category 1 samples for each data point
    M2 = XL[:,CAT2].mean(1) ## Mean of category2 samples for each data point
    V1 = XL[:,CAT1].var(1)  ## Variance of category 1 samples for each data point
    V2 = XL[:,CAT2].var(1)  ## Variance of category 2 samples for each data point
    n1 = len(CAT1)          ## Number of category 1 samples
    n2 = len(CAT2)          ## Number of category 2 samples

    # calculate Z score
    Z = (M1 - M2) / np.sqrt(V1/n1 + V2/n2)

    ## Gaussian distribution CDF, pdf, quantile function, etc.
    from scipy.stats.distributions import norm

    ## The Z-score threshold under a Bonferroni correction
    zst = -norm.ppf(0.025/Z.shape[0])

    # indices of differentially expressed genes
    ii = np.flatnonzero(np.abs(Z) > zst) 
    scores = Z[ii] # scores
    genes = np.array(GID)[ii] # gene names
    annotation = np.array(table.column('peak_annotation'))[ii]
    chr = np.array(table.column('chr'))[ii]
    start = np.array(table.column('start'))[ii]
    end = np.array(table.column('end'))[ii]

    # create new table with differential regions
    newTable = Table().with_columns(['chr', chr, 'start', start, 'end', end,'peak_annotation', annotation, 'symbol', genes, 'score', scores])

    return newTable


This method takes as input the data table we are analyzing, as well as the column names of the first category and second category we would like to compare. For example, let's compare naive and 5 day effector states:

In [ ]:
# get column names of category 1 and 2
CAT1_NAMES = ['N1','N2','N3'] 
CAT2_NAMES = ['E5_1','E5_2','E5_3']

calledPeaksN = callPeaks(table, CAT1_NAMES, CAT2_NAMES)

calledPeaksN

The score column contains how different the peak was between the two categories. Positive values demonstrate peaks more accessible in the first category (i.e. more accessible in naive). Negative values demonstrate peaks more accessible in the second category (i.e. more accessible in effector)

<h2 style="color:red">** Question 3**</h2> 

How many peaks in the table above are more accessible in 5 day effector cells, compared to naive cells?
How many peaks in the table above are less accessible in 5 day effector cells, compared to naive cells? (Save these values as variables, we will be plotting them later)

In [ ]:
# Answer here:



<h2 style="color:red">** Question 4**</h2> 

Repeat this process to find the number of differentially accessible peaks for the transition from E5 to E7.

How many peaks in the table above are more accessible in 7 day effector cells, compared to 5 day effector cells?
How many peaks in the table above are less accessible in 7 day effector cells, compared to 5 day effector cells? (Save these values as variables, we will be plotting them later)

In [ ]:
# Answer



<h2 style="color:red">** Question 5**</h2> 

Repeat this process to find the number of differentially accessible peaks for the transition from E7 to M.

How many peaks in the table above are more accessible in memory cells, compared to 7 day effector cells?
How many peaks in the table above are less accessible in memory cells, compared to 7 day effector cells? (Save these values as variables, we will be plotting them later)

In [ ]:
# Answer


<h2 style="color:red">** Question 6**</h2> 

Make a bar plot that shows shows these changes in accessibility for the transitions N->E5, E5->E7, and E7->M. Plot closed peaks as negative values for easier viewing. There should be 6 total bars on the plot. Make sure to label the axes. (Hint: To make this plot, you can make a new table with the values computed above, or use matplotlib.) If you are unsure of what this plot may sort of look at, reference Figure 1B in the paper for help.

In [ ]:
# Answer


<h2 style="color:red">** Question 7**</h2> 

Given the bar chart above, which transition has the fewest differences in accessibily? Which transition has the most changes in accessibily?

**Answer here:**

    

Now, we will look at some of the genes and observe how their accessibility peaks change over T cell activation.

<h2 style="color:red">** Question 8**</h2> 


In [14]:
# Answer


## Part 2: Comparing Normal and Dysfunction T cell accessibility


Now, we will compare the normal effector states (E5/E7) to dysfunctional T cell states (L5/L7)

<h2 style="color:red">** Question 9**</h2> 

Run the peak calling algorithm, differentiating peaks between normal effector states (E5/E7) and dysfunctional T cell states (L5/L7). Print the resulting table.

In [ ]:
# Answer


<h2 style="color:red">** Question 10**</h2> 


In [15]:
# Answer


## Part 3: Viewing ATAC-seq data in UCSC Genome Browser

Now, we will view a couple accessibility sites for activated T cells in the UCSC genome browser. 

1. Navigate to http://rstats.immgen.org/Chromatin/chromatin.html. 
2. Select ab T cells. Under the ab T cell category, select all of the NK (natural killer) cell categories. There are 4.
![UCSC Home Page](../images/lab4/select_NKcells.png)
3. Search for the gene 'Gzma' and click 'View data on UCSC Genome Browser'
    

<h2 style="color:red">** Question 11**</h2> 
Viewing Gzma in the genome browser, does the naive cell (NKT_Sp) look more or less accessible than the other three activated cell states?

**Answer here:**


<h2 style="color:red">** Question 12**</h2> 
Navigate to gene cards (http://www.genecards.org/) and search for the gene **Gzma** and read the description. What process is this gene involved in? Does the function of this gene align with the accessibility trends you observed in the previous question? Explain.

** Answer here:**


<h2 style="color:red">** Bonus Question**</h2> 
Describe, in words, how the peak calling algorithm works.

** Answer here**:
